In [8]:
from sympy import *
from sympy.solvers.ode.systems import linodesolve

tau_max, omega_max = symbols("tau_max, omega_max", positive=True, nonzero=True) #stall torque, free speed
T1, T2, k, m_a, m_l, k = symbols('T1, T2, k, m_a, m_l, k', positive=True, nonzero=True)
c0, c1, c2, c3, c4, c5, c6 = symbols('c:7')
t = symbols('t')

c1 = -k * T1**2 * omega_max / tau_max
c2 = k * T1 * omega_max / (tau_max * T2)
c3 = k * T1 / (T2 * m_l)
c4 = -k / (T2**2 * m_l)

A = Matrix([
    [c1, c2, 0],
    [0, 0, 1],
    [c3, c4, 0],
])
A

Matrix([
[-T1**2*k*omega_max/tau_max, T1*k*omega_max/(T2*tau_max), 0],
[                         0,                           0, 1],
[             T1*k/(T2*m_l),              -k/(T2**2*m_l), 0]])

In [9]:
b = Matrix([1, 0, 0])
linsol = linodesolve(A, t, b=b)
linsol

[_Dummy_3383/(T1*T2) - _Dummy_3384*T1**3*T2**2*k**2*m_l**(5/2)*omega_max**2*exp(-T1**2*k*omega_max*t/(2*tau_max) - sqrt(k)*t*sqrt(T1**4*T2**2*k*m_l*omega_max**2 - 4*tau_max**2)/(2*T2*sqrt(m_l)*tau_max))/(T1**2*T2*k**2*m_l**(3/2)*omega_max*tau_max + k**(3/2)*m_l*tau_max*sqrt(T1**4*T2**2*k*m_l*omega_max**2 - 4*tau_max**2)) - _Dummy_3384*T1*T2*k**(3/2)*m_l**2*omega_max*sqrt(T1**4*T2**2*k*m_l*omega_max**2 - 4*tau_max**2)*exp(-T1**2*k*omega_max*t/(2*tau_max) - sqrt(k)*t*sqrt(T1**4*T2**2*k*m_l*omega_max**2 - 4*tau_max**2)/(2*T2*sqrt(m_l)*tau_max))/(T1**2*T2*k**2*m_l**(3/2)*omega_max*tau_max + k**(3/2)*m_l*tau_max*sqrt(T1**4*T2**2*k*m_l*omega_max**2 - 4*tau_max**2)) - _Dummy_3385*T1**3*T2**2*k**2*m_l**(5/2)*omega_max**2*exp(-T1**2*k*omega_max*t/(2*tau_max) + sqrt(k)*t*sqrt(T1**4*T2**2*k*m_l*omega_max**2 - 4*tau_max**2)/(2*T2*sqrt(m_l)*tau_max))/(T1**2*T2*k**2*m_l**(3/2)*omega_max*tau_max - k**(3/2)*m_l*tau_max*sqrt(T1**4*T2**2*k*m_l*omega_max**2 - 4*tau_max**2)) + _Dummy_3385*T1*T2*k**(3/2)*m

In [10]:
for row in linsol:
    simprow = simplify(row)
    print(count_ops(simprow))
    display(simprow)

536


(_Dummy_3383*T1**4*T2**2*k**4*m_l**3*omega_max**2*tau_max*exp(t*(T1**2*k*omega_max + sqrt(k)*sqrt(T1**4*T2**2*k*m_l*omega_max**2 - 4*tau_max**2)/(T2*sqrt(m_l)))/(2*tau_max)) - 4*_Dummy_3383*k**3*m_l**2*tau_max**3*exp(t*(T1**2*k*omega_max + sqrt(k)*sqrt(T1**4*T2**2*k*m_l*omega_max**2 - 4*tau_max**2)/(T2*sqrt(m_l)))/(2*tau_max)) - _Dummy_3384*T1**6*T2**4*k**4*m_l**4*omega_max**3 + 4*_Dummy_3384*T1**2*T2**2*k**3*m_l**3*omega_max*tau_max**2 - _Dummy_3385*T1**6*T2**4*k**4*m_l**4*omega_max**3*exp(sqrt(k)*t*sqrt(T1**4*T2**2*k*m_l*omega_max**2 - 4*tau_max**2)/(T2*sqrt(m_l)*tau_max)) + 4*_Dummy_3385*T1**2*T2**2*k**3*m_l**3*omega_max*tau_max**2*exp(sqrt(k)*t*sqrt(T1**4*T2**2*k*m_l*omega_max**2 - 4*tau_max**2)/(T2*sqrt(m_l)*tau_max)) + T1**5*T2**3*k**4*m_l**3*omega_max**2*t*tau_max*exp(t*(T1**2*k*omega_max + sqrt(k)*sqrt(T1**4*T2**2*k*m_l*omega_max**2 - 4*tau_max**2)/(T2*sqrt(m_l)))/(2*tau_max)) - T1**3*T2**2*k**(7/2)*m_l**(5/2)*omega_max*tau_max*sqrt(T1**4*T2**2*k*m_l*omega_max**2 - 4*tau_max**2

KeyboardInterrupt: 

In [11]:
from sympy.solvers.ode.systems import dsolve_system

tau_max, omega_max = symbols("tau_max, omega_max") #stall torque, free speed

T1, T2, k, m_a, m_l = symbols('T1, T2, k, m_a, m_l')
c0, c1, c2, c3, c4, c5, c6 = symbols('c:7')

x_a, x_l = symbols("x_a x_l", cls=Function)
t = symbols('t')

c1 = -k * T1**2 * omega_max / tau_max
c2 = k * T1 * omega_max / (tau_max * T2)
c3 = k * T1 / (T2 * m_l)
c4 = -k / (T2**2 * m_l)

eqs = [
    Eq(x_a(t).diff(t), c1*x_a(t) + c2*x_l(t) + omega_max), 
    Eq(x_l(t).diff(t).diff(t), c3*x_a(t) + c4*x_l(t))
]
display(eqs[0], eqs[1])

# sol = dsolve_system(eqs)
sol = dsolve_system(eqs, ics={x_a(0):0, x_l(0):0, x_l(t).diff(t).subs(t,0):0})
sol

Eq(Derivative(x_a(t), t), -T1**2*k*omega_max*x_a(t)/tau_max + T1*k*omega_max*x_l(t)/(T2*tau_max) + omega_max)

Eq(Derivative(x_l(t), (t, 2)), T1*k*x_a(t)/(T2*m_l) - k*x_l(t)/(T2**2*m_l))

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sympy/simplify/radsimp.py:1131: SymPyDeprecationWarning: 

Using non-Expr arguments in Mul is deprecated (in this case, one of
the arguments has type 'Tuple').

If you really did intend to use a multiplication or addition operation with
this object, use the * or + operator instead.

See https://docs.sympy.org/latest/explanation/active-deprecations.html#non-expr-args-deprecated
for details.

This has been deprecated since SymPy version 1.7. It
will be removed in a future version of SymPy.

  return Mul(*numer, evaluate=not exact), Mul(*denom, evaluate=not exact)


[[Eq(x_a(t), -T1**2*T2*k*m_l*omega_max**2*(T1**8*T2**4*k**2*m_l**2*omega_max**4 + T1**6*T2**3*k*m_l*omega_max**3*sqrt(k*m_l*(T1**4*T2**2*k*m_l*omega_max**2 - 4*tau_max**2)) - 4*T1**4*T2**2*k*m_l*omega_max**2*tau_max**2 - 2*T1**2*T2*omega_max*tau_max**2*sqrt(k*m_l*(T1**4*T2**2*k*m_l*omega_max**2 - 4*tau_max**2)) + 2*tau_max**4)*exp(-t*(T1**2*T2*k*m_l*omega_max + sqrt(T1**4*T2**2*k**2*m_l**2*omega_max**2 - 4*k*m_l*tau_max**2))/(2*T2*m_l*tau_max))*Integral(exp(T1**2*k*omega_max*t/(2*tau_max))*exp(t*sqrt(k*m_l*(T1**4*T2**2*k*m_l*omega_max**2 - 4*tau_max**2))/(2*T2*m_l*tau_max)), (t, 0))/(T1**10*T2**5*k**3*m_l**3*omega_max**5 + T1**8*T2**4*k**2*m_l**2*omega_max**4*sqrt(k*m_l*(T1**4*T2**2*k*m_l*omega_max**2 - 4*tau_max**2)) - 6*T1**6*T2**3*k**2*m_l**2*omega_max**3*tau_max**2 - 4*T1**4*T2**2*k*m_l*omega_max**2*tau_max**2*sqrt(k*m_l*(T1**4*T2**2*k*m_l*omega_max**2 - 4*tau_max**2)) + 8*T1**2*T2*k*m_l*omega_max*tau_max**4 + 2*tau_max**4*sqrt(k*m_l*(T1**4*T2**2*k*m_l*omega_max**2 - 4*tau_max**2))

In [47]:
from sympy.codegen.ast import CodeBlock, Assignment

# print(python(sol[0][0].rhs))

n = Symbol('n')
print(pycode(CodeBlock(Assignment(n, sol[0][0].rhs)).cse()))

  # Not supported in Python:
  # Integral
  # Integral
  # Integral
  # Integral
  # Integral
  # Integral
x0 = T1**2
x1 = omega_max**2
x2 = tau_max**2
x3 = 4*x2
x4 = k*m_l
x5 = x1*x4
x6 = T1**4*T2**2
x7 = -x3 + x5*x6
x8 = math.sqrt(x4*x7)
x9 = 1/x8
x10 = omega_max*x0
x11 = k*x10
x12 = 1/(T2*m_l)
x13 = x12*x8
x14 = (1/2)*t/tau_max
x15 = T2*x10
x16 = x15*x4
x17 = x3*x4
x18 = k**2*m_l**2
x19 = x1*x6
x20 = math.sqrt(-x17 + x18*x19)
x21 = x12*x14
x22 = math.exp(-x21*(x16 + x20))
x23 = math.exp(-x21*(x16 - x20))
x24 = T2*x0
x25 = x15*x8
x26 = math.exp(x11*x14)
x27 = x13*x14
x28 = tau_max**4
x29 = T1**6*T2**3*omega_max**3
x30 = 2*x28
x31 = T1**8*T2**4*omega_max**4*x18
x32 = x17*x19
n = T2*k*m_l*x0*x1*x22*x9*Integral(exp(x14*(x11 + x13)), t) + T2*x0*x1*x23*(x16 + x8)*Integral(x26*exp(-x27), (t, 0))/(x25 + x7) + omega_max*Integral(1, t) - omega_max*Integral(1, (t, 0)) - x22*x24*x5*(-2*x2*x25 + x29*x4*x8 + x30 + x31 - x32)*Integral(x26*exp(x27), (t, 0))/(T1**10*T2**5*k**3*m_l**3*omega_max**5 + 

In [48]:
integ = Integral(exp(t*(T1**2*k*omega_max - sqrt(k*m_l*(T1**4*T2**2*k*m_l*omega_max**2 - 4*tau_max**2))/(T2*m_l))/(2*tau_max)), t)
display(integ, simplify(integ + 1)-1)

Integral(exp(t*(T1**2*k*omega_max - sqrt(k*m_l*(T1**4*T2**2*k*m_l*omega_max**2 - 4*tau_max**2))/(T2*m_l))/(2*tau_max)), t)

Piecewise((2*T2*m_l*tau_max*exp(t*(T1**2*T2*k*m_l*omega_max - sqrt(k*m_l*(T1**4*T2**2*k*m_l*omega_max**2 - 4*tau_max**2)))/(2*T2*m_l*tau_max))/(T1**2*T2*k*m_l*omega_max - sqrt(T1**4*T2**2*k**2*m_l**2*omega_max**2 - 4*k*m_l*tau_max**2)), Ne(T1**2*T2*k*m_l*omega_max - sqrt(T1**4*T2**2*k**2*m_l**2*omega_max**2 - 4*k*m_l*tau_max**2), 0)), (t, True))

In [197]:
x_a_eq = simplify(sol[0][0])
x_l_eq = simplify(sol[0][1])
display(x_a_eq, x_l_eq)

Eq(x_a(t), -T1**2*T2*k*m_l*omega_max**2*(T1**8*T2**4*k**2*m_l**2*omega_max**4 + T1**6*T2**3*k*m_l*omega_max**3*sqrt(k*m_l*(T1**4*T2**2*k*m_l*omega_max**2 - 4*tau_max**2)) - 4*T1**4*T2**2*k*m_l*omega_max**2*tau_max**2 - 2*T1**2*T2*omega_max*tau_max**2*sqrt(k*m_l*(T1**4*T2**2*k*m_l*omega_max**2 - 4*tau_max**2)) + 2*tau_max**4)*Piecewise((2*T2*m_l*tau_max/(T1**2*T2*k*m_l*omega_max + sqrt(T1**4*T2**2*k**2*m_l**2*omega_max**2 - 4*k*m_l*tau_max**2)), Ne(k, 0)), (0, True))*exp(-t*(T1**2*T2*k*m_l*omega_max + sqrt(T1**4*T2**2*k**2*m_l**2*omega_max**2 - 4*k*m_l*tau_max**2))/(2*T2*m_l*tau_max))/(T1**10*T2**5*k**3*m_l**3*omega_max**5 + T1**8*T2**4*k**2*m_l**2*omega_max**4*sqrt(k*m_l*(T1**4*T2**2*k*m_l*omega_max**2 - 4*tau_max**2)) - 6*T1**6*T2**3*k**2*m_l**2*omega_max**3*tau_max**2 - 4*T1**4*T2**2*k*m_l*omega_max**2*tau_max**2*sqrt(k*m_l*(T1**4*T2**2*k*m_l*omega_max**2 - 4*tau_max**2)) + 8*T1**2*T2*k*m_l*omega_max*tau_max**4 + 2*tau_max**4*sqrt(k*m_l*(T1**4*T2**2*k*m_l*omega_max**2 - 4*tau_max**2)

Eq(x_l(t), -2*T1*T2*k*m_l*omega_max*tau_max**2*(T1**8*T2**4*k**2*m_l**2*omega_max**4 + T1**6*T2**3*k*m_l*omega_max**3*sqrt(k*m_l*(T1**4*T2**2*k*m_l*omega_max**2 - 4*tau_max**2)) - 4*T1**4*T2**2*k*m_l*omega_max**2*tau_max**2 - 2*T1**2*T2*omega_max*tau_max**2*sqrt(k*m_l*(T1**4*T2**2*k*m_l*omega_max**2 - 4*tau_max**2)) + 2*tau_max**4)*Piecewise((2*T2*m_l*tau_max/(T1**2*T2*k*m_l*omega_max + sqrt(T1**4*T2**2*k**2*m_l**2*omega_max**2 - 4*k*m_l*tau_max**2)), Ne(k, 0)), (0, True))*exp(-t*(T1**2*T2*k*m_l*omega_max + sqrt(T1**4*T2**2*k**2*m_l**2*omega_max**2 - 4*k*m_l*tau_max**2))/(2*T2*m_l*tau_max))/((T1**2*T2*k*m_l*omega_max + sqrt(k*m_l*(T1**4*T2**2*k*m_l*omega_max**2 - 4*tau_max**2)))*(T1**10*T2**5*k**3*m_l**3*omega_max**5 + T1**8*T2**4*k**2*m_l**2*omega_max**4*sqrt(k*m_l*(T1**4*T2**2*k*m_l*omega_max**2 - 4*tau_max**2)) - 6*T1**6*T2**3*k**2*m_l**2*omega_max**3*tau_max**2 - 4*T1**4*T2**2*k*m_l*omega_max**2*tau_max**2*sqrt(k*m_l*(T1**4*T2**2*k*m_l*omega_max**2 - 4*tau_max**2)) + 8*T1**2*T2*k*m

In [5]:
x_l_eq.rhs

NameError: name 'x_l_eq' is not defined

In [224]:
x_l_expr = x_l_eq.rhs
x_l_refined = refine(x_l_expr, Q.nonzero(k))

KeyboardInterrupt: 

In [223]:
from sympy.codegen.ast import CodeBlock, Assignment

x_a_expr = (x_a_eq.rhs)
x_l_expr = (x_l_eq.rhs)

x_acode = symbols('x_acode')
x_lcode = symbols('x_lcode')
c = CodeBlock(Assignment(x_lcode, x_l_expr))

print("operations: ", count_ops(c.cse()))

print(ccode(c.cse()))

operations:  154
x0 = T1*T2*omega_max;
x1 = pow(tau_max, 2);
x2 = 4*x1;
x3 = k*m_l;
x4 = pow(T1, 4)*pow(T2, 2)*pow(omega_max, 2);
x5 = -x2 + x3*x4;
x6 = sqrt(x3*x5);
x7 = pow(T1, 2)*omega_max;
x8 = T2*x7;
x9 = x6*x8;
x10 = x3*x8;
x11 = x10 + x6;
x12 = x2*x3;
x13 = pow(k, 2)*pow(m_l, 2);
x14 = sqrt(-x12 + x13*x4);
x15 = x10 - x14;
x16 = 2*T2*m_l*tau_max;
x17 = x16/x15;
x18 = k != 0;
x19 = 1/(T2*m_l);
x20 = (1.0/2.0)*t/tau_max;
x21 = x19*x20;
x22 = 2*x1;
x23 = x0*x22;
x24 = x23*exp(-x15*x21)/(x10 - x6);
x25 = k*x7;
x26 = x19*x6;
x27 = x10 + x14;
x28 = x16/x27;
x29 = x3/x6;
x30 = x23*exp(-x21*x27)/x11;
x31 = pow(tau_max, 4);
x32 = pow(T1, 6)*pow(T2, 3)*pow(omega_max, 3);
x33 = 2*x31;
x34 = pow(T1, 8)*pow(T2, 4)*pow(omega_max, 4)*x13;
x35 = x12*x4;
x_lcode = t*x0 + x11*x24*((x18) ? (
   x17
)
: (
   0
))/(x5 + x9) - x24*x29*((x15 != 0) ? (
   x17*exp(x20*(x25 - x26))
)
: (
   t
)) + x29*x30*((x27 != 0) ? (
   x28*exp(x20*(x25 + x26))
)
: (
   t
)) - x3*x30*(-x22*x9 + x3*x32*x6 + x33 + x34 

In [204]:
print(pycode(c.cse()))

x0 = T1*T2*omega_max
x1 = tau_max**2
x2 = 4*x1
x3 = k*m_l
x4 = T1**4*T2**2*omega_max**2
x5 = -x2 + x3*x4
x6 = math.sqrt(x3*x5)
x7 = T1**2*omega_max
x8 = T2*x7
x9 = x6*x8
x10 = x3*x8
x11 = x10 + x6
x12 = x2*x3
x13 = k**2*m_l**2
x14 = math.sqrt(-x12 + x13*x4)
x15 = x10 - x14
x16 = 2*T2*m_l*tau_max
x17 = x16/x15
x18 = (k != 0)
x19 = 1/(T2*m_l)
x20 = (1/2)*t/tau_max
x21 = x19*x20
x22 = 2*x1
x23 = x0*x22
x24 = x23*math.exp(-x15*x21)/(x10 - x6)
x25 = k*x7
x26 = x19*x6
x27 = x10 + x14
x28 = x16/x27
x29 = x3/x6
x30 = x23*math.exp(-x21*x27)/x11
x31 = tau_max**4
x32 = T1**6*T2**3*omega_max**3
x33 = 2*x31
x34 = T1**8*T2**4*omega_max**4*x13
x35 = x12*x4
x_lcode = t*x0 + x11*x24*((x17) if x18 else (0))/(x5 + x9) - x24*x29*((x17*math.exp(x20*(x25 - x26))) if (x15 != 0) else (t)) + x29*x30*((x28*math.exp(x20*(x25 + x26))) if (x27 != 0) else (t)) - x3*x30*(-x22*x9 + x3*x32*x6 + x33 + x34 - x35)*((x28) if x18 else (0))/(T1**10*T2**5*k**3*m_l**3*omega_max**5 - 6*x1*x13*x32 + 8*x10*x31 + x33*x6 + x34*x6 